## Loading Data


In [2]:

import pandas as pd
import seaborn as sns


df_items = pd.read_csv(r'C:\Users\kkaus\OneDrive - HKUST Connect\Foodpanda\Data\item_list.csv')
df_outlets = pd.read_csv(r'C:\Users\kkaus\OneDrive - HKUST Connect\Foodpanda\Data\outlet_visits.csv')
food_database = pd.read_csv(r"C:\Users\kkaus\OneDrive\Documents\GitHub\foodpanda-project\data\generic-food.csv")

df_items.shape
food_database.shape



(906, 4)

### TODO:
Clean data: 
 - Numbers in Dish Name: "1107. Lamb Kadai"
 - nan in primary_cuisine
 - tyr without including primary cuisine


 Primary Cuisine:
  - 1 per vendor - not dish

80% Training, 20% Testing, no validation to allow for better accuracy
 - Doesn't apply, don't split
 -

sklearn train test split

prepocessing:
x is concatednated list of all dishes and resto name, y is all the primary cuisine, remove all the bad rows, labels are unique values


ingreds, secondary is unsupervised learning, don't have accuracy numbers
confusion matrix for primary

## Inspecting data

In [ ]:
df_items

In [ ]:
scrape_df = df_items[["title_english"]]

In [ ]:
scrape_df = scrape_df.sample(30)

In [ ]:
scrape_df.to_csv("scrape.csv", header=True, index=False)


Out of 718,342 dishes 718,284 have cuisines, so we can generalize based of that



In [ ]:
df_items.isnull().sum()

In [3]:
df_items.nunique()

brand_name               6861
outlet_name             12069
product_title          294500
title_english          293885
description             50151
description_english     49970
primary_cuisine            41
cuisines                 3368
Sep Orders               2844
Oct Orders               2858
Nov Orders               2868
dtype: int64

In [ ]:
filter = df_items["brand_name"] == "Caf"
la_pizza = df_items[filter].to_string(header=False, index=False)
la_pizza

In [ ]:
df_items[filter]

In [4]:
df_items.primary_cuisine.unique()

# remove whole rows, don't hard code,if it more than 10%, then do manual labeling

array(['Middle Eastern', 'Malaysian', nan, 'Singaporean', 'German',
       'French', 'Indonesian', 'Mexican', 'Asian', 'Fast Food', 'Dessert',
       'Healthy', 'Spanish', 'Mediterranean', 'Beijing', 'European',
       'British', 'Drinks', 'Pizza', 'Burgers', 'Sandwich',
       'South American', 'Halal', 'Noodles', 'Cakes & Bakery', 'Thai',
       'Indian', 'Korean', 'Chinese', 'Italian', 'Sichuan', 'Western',
       'American', 'Japanese', 'Cantonese', 'Chiu Chow', 'Hong Kong',
       'Taiwanese', 'Vietnamese', 'Shanghainese', 'International',
       'Southeast Asian'], dtype=object)

In [ ]:
len(df_items.primary_cuisine.unique())

In [ ]:
df_items

In [2]:
df_clean = df_items.drop(["Fast Food", "Dessert", "Healthy", "Drinks", "Pizza", "Burgers", "Sandwich", "Noodles", "Cakes & Bakery"], axis = 0, inplace = True)

KeyError: "['Fast Food' 'Dessert' 'Healthy' 'Drinks' 'Pizza' 'Burgers' 'Sandwich'\n 'Noodles' 'Cakes & Bakery'] not found in axis"

# Zero Shot Classification

In [4]:
!pip install pytest

In [5]:
from transformers import pipeline

# Set up
classifier = pipeline("zero-shot-classification", model='typeform/distilbert-base-uncased-mnli')

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'random' from partially initialized module 'tensorflow._api.v2.compat.v2.experimental.numpy' (most likely due to a circular import) (c:\Users\kkaus\OneDrive - HKUST Connect\Foodpanda\Data\foodpanda\lib\site-packages\tensorflow\_api\v2\compat\v2\experimental\numpy\__init__.py)

# 1. Primary Cuisine Zero-Shot  (PER VENDOR) 

#### Note: only put dish name and restaurant name, all dish names of one restuarant


In [ ]:
def primary_cuisine(sequence, labels, target):
       global output, correct
       output = classifier(sequence, labels)

       print("1. Primary Cuisine:")
       print("")
       print("Input:", output['sequence'])
       print("")
       print("")
       print("")
       print("Primary Cuisine:", output['labels'][0])
       print("Certainty:", output['scores'][0])
       print("")
       print("")
       print("")
       print("Target:",target)
       print("")


       if output['labels'][0] == target:
              correct = True
       
primary_cuisine_country_labels = ['Middle Eastern', 'Malaysian', 'Singaporean', 'German',
       'French', 'Indonesian', 'Mexican', 'Spanish', 'Mediterranean', 'Beijing', 'European',
       'British', 'South American','Thai', 'Indian', 'Korean', 'Chinese', 'Italian', 'American', 'Japanese', 'Hong Kong',
       'Taiwanese', 'Vietnamese', 'International',
       'Southeast Asian']

# itterate over every restaurant

In [ ]:
import random

index = random.randint(0,718342)
sample = df_items.iloc[index:index+1, [0,1,2,3,4,5]].to_string(header=False, index=False)
target = df_items.iloc[[index]]["primary_cuisine"].to_string(header=False, index=False)
primary_cuisine(sample, primary_cuisine_country_labels, target)


In [ ]:
output

# 2. Secondary Cuisine Zero-Shot (3 PER VENDOR)

#### Dishes of all restors


In [ ]:
# secondary_cuisine_list = ["Sushi", "Pizza", "Pasta", "Noodle", "BBQ", "Burger", "Dumpling", "Curry", "Cake", "Taco", "Burrito"]

dishes = food_database["GROUP"] == "Dishes"     #Can also include beverages, teas
secondary_cuisine_list = food_database[dishes]["FOOD NAME"].to_list()


def secondary_cuisine(sequence, labels):
    global output
    output = classifier(sequence, labels, multi_label = True)

    print("2. Secondary Cuisines:")
    print("")
    print("Vendor Name (Input):" , output['sequence'])
    print("")
    print("")
    print("")
    print("Output:")
    print("Secondary Cuisine Tag 1:", output['labels'][0], "({})".format(output['scores'][0]))
    print("Secondary Cuisine Tag 2:", output['labels'][1], "({})".format(output['scores'][1]))
    print("Secondary Cuisine Tag 3:", output['labels'][2], "({})".format(output['scores'][2]))
    print("")




In [ ]:
index = random.randint(0,718342)
seq = df_items.iloc[[index]].to_string(header=False, index=False)
secondary_cuisine(seq, secondary_cuisine_list)

# 3. Suitability (2 per vendor)

In [ ]:
suitability_list = ["Lactose", "Gluten", "Vegetarian", "Vegan", "Kosher", "Keto", "Dairy" , "Nuts", "Wheat"]

def suitability(sequence, labels):
    global output
    output = classifier(sequence, labels, multi_label = True)

    print("3. Suitability:")
    print("")
    print("Vendor Name (Input):" , output['sequence'])
    print("")
    print("")
    print("")
    print("Output:")
    print("Suitability Tag 1:", output['labels'][0], "({})".format(output['scores'][0]))
    print("Suitability Tag 2:", output['labels'][1], "({})".format(output['scores'][1]))
    print("")




In [ ]:
index = random.randint(0,718342)
seq = df_items.iloc[[index]].to_string(header=False, index=False)
suitability(seq, suitability_list)

# 4. Primary Ingredient Zero-Shot (5 PER DISH)

In [ ]:

ingredients_list = ['Beef', 'Pork', 'Chicken', 'Rice', 'Pasta', 'Fish', 'Udon', 'Vegetables', 'Spinach', 'X', 'Eggs', 'Lamb', 'Noodles', 'Salmon', 'Tuna'] 
# Need to expand list like crazy

# output = classifier(sample, ingredients_list, multi_label=True)

def ingredients(sequence, labels):
       global output
       output = classifier(sequence, labels, multi_label=True)

       print("4. Main Ingredients:")
       print("")
       print("Input:", output['sequence'])
       print("")
       print("")
       print("")
       print("Output:")
       print("Ingredient 1:", output['labels'][0])
       print("Ingredient 2:", output['labels'][1])
       print("Ingredient 3:", output['labels'][2])
       print("Ingredient 4:", output['labels'][3])
       print("Ingredient 5:", output['labels'][4])
       print("Certainty:", output['scores'][0])
       print("")


In [ ]:
output

In [ ]:
index = random.randint(0,718342)
sample = df_items.iloc[[index]].to_string(header=False, index=False)
ingredients(sample, ingredients_list)

In [ ]:
output

# 5. Taste Tags Zero-Shot (5 per dish)


In [ ]:
tastes = ['Sweet', 'Salty', 'Sour', 'Bitter', 'Umami', 'Fat', 'Spicy', 'Numbing']

def taste(sequence, labels):
       global output
       output = classifier(sequence, labels, multi_label=True)

       print("5. Taste:")
       print("")
       print("Input:", output['sequence'])
       print("")
       print("")
       print("")
       print("Output:")
       print("Ingredient 1:", output['labels'][0])
       print("Ingredient 2:", output['labels'][1])
       print("Ingredient 3:", output['labels'][2])
       print("Ingredient 4:", output['labels'][3])
       print("Ingredient 5:", output['labels'][4])
       print("Certainty:", output['scores'][0])
       print("")


In [ ]:
index = random.randint(0,718342)
sample = df_items.iloc[[index]].to_string(header=False, index=False)
taste(sample, tastes)

# All Tags Displayed per Dish

In [ ]:
index = random.randint(0,718342)
sample = df_items.iloc[index:index+1, [0,1,2,3,4,5]].to_string(header=False, index=False)
primary_cuisine(sample, primary_cuisine_country_labels, target)
secondary_cuisine(sample, secondary_cuisine_list)
suitability(sample, suitability_list)
ingredients(sample, ingredients_list)
taste(sample, tastes)

#### Accuracy demo

In [ ]:
df_items = df_items.dropna()

In [ ]:
X = df_items.iloc[:, [0,1,2,3,4,5]].values.tolist()
y = df_items.iloc[:, [6]].values.tolist()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)